<a href="https://colab.research.google.com/github/fatemafaria142/Comparative-Analysis-of-Diverse-Large-Language-Models-in-Story-Generation/blob/main/Story_Generation_using_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00


### **Dataset Link:** https://huggingface.co/datasets/AtlasUnified/atlas-storyteller

In [ ]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**

* The dataset contains three different columns.

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})

In [ ]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("ID:", data['id'])
    print("Story:", data['Story'])
    print("\n-----------------------------\n")


Data Point 1:
ID: seed_task_0
Story: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room

### **We will use just a small subset of the data for this training example**

In [ ]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(1500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(200))

In [ ]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['id', 'Story'],
        num_rows: 200
    })
})

In [ ]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    eos_token = "</s>"

    # Use a predefined template for instructions
    instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"



    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instructions:"
    full_prompt += "\n" + instructions_template
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + sample["Story"]
    full_prompt += "\n\n### Response:"
    full_prompt += eos_token

    return full_prompt


In [ ]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>### Instructions:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!\n\n### Input:\nIn the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were 

In [ ]:
create_prompt(instruct_tune_dataset["train"][1])

'<s>### Instructions:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!\n\n### Input:\nThe sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. Among the pilots was Captain Joshua Bennett, a seasoned veteran with eyes as sharp as a hawk\'s, and a heart that beat with the rhythm of war.  Set in the midst of a raging conflict, the story takes place during the height of World War II. Captain Bennett, a fearless and daring leader, had earned his reputation through countless successful missions. He was known for his audacious maneuvers and masterful control of aircraft, traits that instilled confidence in 

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [ ]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**
* `temperature=0.5` sets a moderate level of randomness. You can experiment with different values for temperature to achieve the desired balance between creativity and determinism in your generated text. Adjust the value based on your specific use case and preferences.

In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024,temperature=0.5, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [ ]:
prompt="### Instruction:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!\n\n###Input:In the small rural town of Meadowbrook, nestled amidst rolling green hills and vast fields of wheat, life was peaceful, if unremarkable.\n\n### Response:"

In [ ]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s> \nBut all that changed when a mysterious stranger arrived in town. He was tall, thin, and dressed in black, and he carried with him a small, leather-bound book. No one knew where he came from, or why he chose to settle in Meadowbrook, but they couldn't deny that he was intriguing.\n\nThe stranger was soon the talk of the town. Some speculated that he was a spy, while others believed he was a writer. But one thing was certain - he was a loner, and he kept to himself.\n\nDespite his reclusive nature, the stranger was a skilled craftsman. He had a workshop in the old mill at the edge of town, and he spent his days working on intricate wooden puzzles and games. No one knew what he was working on, but they couldn't help but be fascinated by his creations.\n\nOne day, a group of children gathered outside the workshop, curious about what the stranger was up to. They peered through the windows, watching as he worked on a large wooden box. Suddenly, the box sprang open, revealing a small, 

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [ ]:
from transformers import TrainingArguments
output_model= "mistral_biomedical_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=50,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [ ]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.616400
20,1.438300
30,1.363300
40,1.347500
50,1.340000


TrainOutput(global_step=50, training_loss=1.4210812950134277, metrics={'train_runtime': 750.7723, 'train_samples_per_second': 0.266, 'train_steps_per_second': 0.067, 'total_flos': 8741766719078400.0, 'train_loss': 1.4210812950134277, 'epoch': 0.13})

### **Save the model**

In [ ]:
trainer.save_model("mistral_biomedical_generation")

In [ ]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024,temperature=0.5, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example: 1**

In [ ]:
# Example usage
prompt = "### Instruction:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
prompt += "\n### Input:\nOnce upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> ### Instruction:
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!
### Input:
Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.

### Response:
As the years passed, the three friends grew up and their bond only strengthened. They shared their hopes, dreams, and fears with each other, and always had each other's backs.

One day, while wandering through the streets of Dhaka, they stumbled upon an old bookstore that they had never been to before. As they browsed through the dusty shelves, they found a book that caught their attention - "The Lost City of Gold."

Excited by the prospect of uncovering the secrets of the lost city,

### **Example: 2**

In [ ]:
# Example usage
prompt = "### Instruction:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
prompt += "\n### Input:\nIn Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!
### Input:
In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams.

### Response:
As time passed, the trio's friendship grew stronger, and they pursued their passions with fervor. Asif, a brilliant computer scientist, dreamed of creating a revolutionary new software that would change the world. Mukaffi, an aspiring engineer, envisioned a future where renewable energy was accessible to all. Morshed, a budding physicist, longed to unlock the secrets of the universe and unravel the mysteries of quantum mechanics.

One day, while working on their latest projects, the trio stumbled up

### **Example: 3**

In [ ]:
# Example usage
prompt = "### Instruction:\nImagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
prompt += "\n### Input:\nIn a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive."
prompt += "\n\n### Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instruction:
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!
### Input:
In a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive.

### Response:
One day, while Fasabbih was working on his latest project, Mariya received a mysterious phone call. The voice on the other end of the line was soft and gentle, but the words that came out of Mariya's mouth were filled with fear.

"Mariya, listen carefully," the voice said. "I have a message for you. Your brother is in danger, and you must do everything in your power to protect him."

Mariya hung up the phon